In [ ]:
# !pip install pytorch-transformers
# !pip install evaluate
# !pip install wandb
# !pip install datasets

In [ ]:
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, DatasetDict
import time
import torch.nn as nn
import os
os.environ["WANDB_DISABLED"] = "true"
import random
from pytorch_transformers import BertTokenizer, BertConfig
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm
import io
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    TrainerCallback
)
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
stop_words_en = set(stopwords.words('english'))
stemmer_en = SnowballStemmer('english')
from IPython.display import clear_output
import torch.nn.functional as F
import warnings
from torch.nn.parameter import Parameter
from torch.nn.utils import weight_norm
from transformers import get_linear_schedule_with_warmup
from torch.optim.lr_scheduler import LinearLR

warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print("CUDA is available?", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


CUDA is available? True


In [ ]:
file_path = "/content/drive/MyDrive/IMDB Dataset.csv"
data = pd.read_csv(file_path)
data['sentiment'] = data['sentiment'].map({'negative': 0, 'positive': 1})
data = data.drop_duplicates()
# data = data.head(10000)
train_df, test_df = train_test_split(data, test_size=0.3, random_state=42)

data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


In [ ]:
teacher_model_name = "ilkonz/all_imdb_bert" # "ilkonz/bert_imdb_model"
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_name)
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)
teacher_model.eval()

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

class IMDBDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=128):
        self.texts = dataframe['review'].tolist()
        self.labels = dataframe['sentiment'].tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.texts[idx], padding='max_length',
                                  truncation=True, max_length=self.max_length,
                                  return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

batch_size = 32
train_dataset = IMDBDataset(train_df, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Попытка 2

In [ ]:
teacher_model = AutoModelForSequenceClassification.from_pretrained(
    "ilkonz/all_imdb_bert",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=True
)

In [ ]:
class ZeroOp(nn.Module):
    def forward(self, x):
        return torch.zeros_like(x)

class SearchCell(nn.Module):
    def __init__(self, hidden_size=256):
        super().__init__()
        self.hidden_size = hidden_size

        # наше пространство поиска
        self.ops = nn.ModuleDict({
            'conv1': nn.Sequential(
                nn.ReLU(),
                nn.Conv1d(hidden_size, hidden_size, kernel_size=1),
                nn.BatchNorm1d(hidden_size)
            ),
            'conv3': nn.Sequential(
                nn.ReLU(),
                nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1),
                nn.BatchNorm1d(hidden_size)
            ),
            'conv5': nn.Sequential(
                nn.ReLU(),
                nn.Conv1d(hidden_size, hidden_size, kernel_size=5, padding=2),
                nn.BatchNorm1d(hidden_size)
            ),
            'dil_conv3': nn.Sequential(
                nn.ReLU(),
                nn.Conv1d(hidden_size, hidden_size, kernel_size=3, dilation=2, padding=2),
                nn.BatchNorm1d(hidden_size)
            ),
            'max_pool': nn.Sequential(
                nn.ConstantPad1d((1, 1), 0),
                nn.MaxPool1d(kernel_size=3, stride=1)
            ),
            'avg_pool': nn.Sequential(
                nn.ConstantPad1d((1, 1), 0),
                nn.AvgPool1d(kernel_size=3, stride=1)
            ),
            'skip': nn.Identity()
        })

    def forward(self, x, arch_weights):
        results = []
        for op in self.ops.values():
            results.append(op(x))

        results = torch.stack(results)
        arch_weights = arch_weights.view(-1, 1, 1, 1)
        return (results * arch_weights).sum(0)

In [ ]:
# class AdaBERT(nn.Module):
#     def __init__(self, vocab_size, hidden_size=256, max_layers=8):
#         super().__init__()
#         self.hidden_size = hidden_size
#         self.max_layers = max_layers

#         # дергаем эмбеддинги
#         self.embedding = nn.Embedding(vocab_size, hidden_size)

#         # параметры архитектуры
#         self.layer_weights = nn.Parameter(torch.ones(max_layers))
#         self.op_weights = nn.Parameter(torch.ones(len(SearchCell().ops)))

#         # ячейки
#         self.cells = nn.ModuleList([SearchCell(hidden_size) for _ in range(max_layers)])

#         # классификатор
#         self.classifier = nn.Linear(hidden_size, 2)

#         # параметры для efficiency loss
#         self.register_buffer('op_flops', torch.tensor([
#             3 * hidden_size * hidden_size,  # conv3
#             5 * hidden_size * hidden_size,   # conv5
#             3 * hidden_size * hidden_size,   # dil_conv3
#             hidden_size,                    # max_pool
#             hidden_size,                    # avg_pool
#             0,                              # skip
#             0
#         ], dtype=torch.float32))

#         self.register_buffer('op_params', torch.tensor([
#             3 * hidden_size * hidden_size + 2 * hidden_size,
#             5 * hidden_size * hidden_size + 2 * hidden_size,
#             3 * hidden_size * hidden_size + 2 * hidden_size,
#             0,
#             0,
#             0,
#             0
#         ], dtype=torch.float32))

#     def efficiency_loss(self):
#         op_probs = F.softmax(self.op_weights, dim=-1)
#         layer_probs = F.softmax(self.layer_weights, dim=-1)

#         expected_flops = (op_probs * self.op_flops).sum() * layer_probs.sum()
#         expected_params = (op_probs * self.op_params).sum() * layer_probs.sum()

#         total_params = expected_params + sum(p.numel() for p in self.embedding.parameters()) + sum(p.numel() for p in self.classifier.parameters())

#         max_flops = self.op_flops.max() * self.max_layers
#         max_params = self.op_params.max() * self.max_layers + sum(p.numel() for p in self.embedding.parameters()) + sum(p.numel() for p in self.classifier.parameters())

#         norm_flops = expected_flops / max_flops
#         norm_params = total_params / max_params

#         return norm_flops + norm_params

#     def forward(self, input_ids):
#         x = self.embedding(input_ids).transpose(1, 2)
#         layer_probs = F.gumbel_softmax(self.layer_weights, tau=1, hard=False)
#         op_probs = F.gumbel_softmax(self.op_weights, tau=1, hard=False)

#         hidden_states = []
#         for i in range(self.max_layers):
#             if layer_probs[i] > 0.1:
#                 x = self.cells[i](x, op_probs)
#                 hidden_states.append(x)

#         x = x.mean(dim=-1)
#         return self.classifier(x), hidden_states


In [ ]:
# Класс AdaBERT — это сжимаемая модель, в которой преобразуются слова в вектора,
# в ней реализуется проход через набор ячеек, где каждая ячейка сама выбирает одну операцию,
# выбирается, какие ячейки использовать, с помощью обучаемых весов (layer_weights)
# Для каждой ячейки выбирает операцию с помощью Gumbel-Softmax (op_weights)
# Накладывается штраф за тяжелую архитектуру
# В конце усредняет выход и предсказывает класс отзыва (положительный / отрицательный)

class AdaBERT(nn.Module):
    def __init__(self, vocab_size, hidden_size=256, max_layers=8):
        super().__init__()
        self.hidden_size = hidden_size
        self.max_layers = max_layers

        self.embedding = nn.Embedding(vocab_size, hidden_size)

        self.layer_weights = nn.Parameter(torch.ones(max_layers))
        self.op_weights = nn.ParameterList([
            nn.Parameter(torch.ones(len(SearchCell(hidden_size).ops))) for _ in range(max_layers)
        ])

        self.cells = nn.ModuleList([SearchCell(hidden_size) for _ in range(max_layers)])
        self.classifier = nn.Linear(hidden_size, 2)

        self.register_buffer('op_flops', torch.tensor([
            1 * hidden_size * hidden_size,
            3 * hidden_size * hidden_size,
            5 * hidden_size * hidden_size,
            3 * hidden_size * hidden_size,
            hidden_size,
            hidden_size,
            0
        ], dtype=torch.float32))

        self.register_buffer('op_params', torch.tensor([
            1 * hidden_size * hidden_size + 2 * hidden_size,
            3 * hidden_size * hidden_size + 2 * hidden_size,
            5 * hidden_size * hidden_size + 2 * hidden_size,
            3 * hidden_size * hidden_size + 2 * hidden_size,
            0,
            0,
            0
        ], dtype=torch.float32))

    def efficiency_loss(self):
        total_flops = 0
        total_params = 0
        layer_probs = F.softmax(self.layer_weights, dim=-1)

        for i in range(self.max_layers):
            op_probs = F.softmax(self.op_weights[i], dim=-1)
            flops = (op_probs * self.op_flops).sum()
            params = (op_probs * self.op_params).sum()
            total_flops += flops * layer_probs[i]
            total_params += params * layer_probs[i]

        total_params += sum(p.numel() for p in self.embedding.parameters()) + \
                        sum(p.numel() for p in self.classifier.parameters())

        return total_flops / 1e6 + total_params / 1e6

    def forward(self, input_ids):
        x = self.embedding(input_ids).transpose(1, 2)
        layer_probs = F.gumbel_softmax(self.layer_weights, tau=1, hard=True)

        hidden_states = []
        for i in range(self.max_layers):
            if layer_probs[i] > 0:
                op_probs = F.gumbel_softmax(self.op_weights[i], tau=1, hard=True)
                x = self.cells[i](x, op_probs)
                hidden_states.append(x)

        x = x.mean(dim=-1)
        return self.classifier(x), hidden_states

По сути, тут это функция, которая учит ученика копировать hidden state учителя, чтобы ученик стал умнее, даже если у него меньше параметров.

In [ ]:
def custom_kd_loss(student_logits, teacher_hidden_states, student_hidden, layer_mapping):
    loss = 0
    for s_layer, t_layer in layer_mapping.items():
        if s_layer >= len(student_hidden) or t_layer >= len(teacher_hidden_states):
            continue

        s_hidden = student_hidden[s_layer]
        t_hidden = teacher_hidden_states[t_layer]

        if s_hidden.size(1) != t_hidden.size(2):
            t_hidden = t_hidden.transpose(1, 2)

        if s_hidden.size(1) != t_hidden.size(1):
            proj = nn.Linear(s_hidden.size(1), t_hidden.size(1)).to(s_hidden.device)
            s_hidden = proj(s_hidden.transpose(1, 2)).transpose(1, 2)

        loss += F.mse_loss(s_hidden, t_hidden)

    return loss / len(layer_mapping) if len(layer_mapping) > 0 else 0

In [ ]:
def calculate_accuracy(logits, labels):
    preds = torch.argmax(logits, dim=1)
    return (preds == labels).float().mean().item()

class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

def train_adabert():

    tokenizer = AutoTokenizer.from_pretrained("ilkonz/all_imdb_bert")
    train_dataset = TextDataset(train_df['review'].values, train_df['sentiment'].values, tokenizer)
    test_dataset = TextDataset(test_df['review'].values, test_df['sentiment'].values, tokenizer)

    # Даталоадеры
    train_dataloader = DataLoader(train_dataset, batch_size=16)
    test_dataloader = DataLoader(test_dataset, batch_size=16)

    teacher_model = AutoModelForSequenceClassification.from_pretrained(
        "ilkonz/all_imdb_bert",
        num_labels=2,
        output_hidden_states=True
    ).to('cuda')

    student = AdaBERT(len(tokenizer.vocab)).to('cuda')

    optimizer = torch.optim.AdamW(student.parameters(), lr=5e-5)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_dataloader)*5)

    # Обучение
    best_acc = 0
    for epoch in range(15):
        student.train()
        epoch_loss = 0
        epoch_acc = 0

        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/5"):
            batch = {k: v.to('cuda') for k, v in batch.items()}

            # Forward passes
            teacher_outputs = teacher_model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask']
            )
            student_logits, student_hidden = student(batch['input_ids'])

            ce_loss = F.cross_entropy(student_logits, batch['labels'])
            kd_loss = custom_kd_loss(
                student_logits,
                teacher_outputs.hidden_states,
                student_hidden,
                {0:0, 1:3, 2:6, 3:9}
            )
            eff_loss = student.efficiency_loss()
            loss = 0.2*ce_loss + 0.8*kd_loss + 1.5*eff_loss

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # scheduler.step() можно с ним, но без него гораздо лучше

            epoch_loss += loss.item()
            epoch_acc += calculate_accuracy(student_logits, batch['labels'])

        student.eval()
        test_acc = 0
        with torch.no_grad():
            for batch in tqdm(test_dataloader, desc="Validating"):
                batch = {k: v.to('cuda') for k, v in batch.items()}
                logits, _ = student(batch['input_ids'])
                test_acc += calculate_accuracy(logits, batch['labels'])

        print(f"\nEpoch {epoch+1}:")
        print(f"Train Loss: {epoch_loss/len(train_dataloader):.4f}")
        print(f"Train Acc: {epoch_acc/len(train_dataloader):.4f}")
        print(f"Test Acc: {test_acc/len(test_dataloader):.4f}")

        # Сохранение лучшей модели
        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(student.state_dict(), "best_adabert.pth")
            print("Saved new best model")

    print(f"\nBest Validation Accuracy: {best_acc/len(test_dataloader):.4f}")

train_adabert()

Validating: 100%|██████████| 938/938 [00:22<00:00, 42.57it/s]



Epoch 1:
Train Loss: 12.3098
Train Acc: 0.5308
Test Acc: 0.5556
Saved new best model


Validating: 100%|██████████| 938/938 [00:21<00:00, 42.65it/s]



Epoch 2:
Train Loss: 12.2820
Train Acc: 0.5766
Test Acc: 0.5983
Saved new best model


Validating: 100%|██████████| 938/938 [00:22<00:00, 42.33it/s]


...


Epoch 13:
Train Loss: 12.1264
Train Acc: 0.7072
Test Acc: 0.7082
Saved new best model


Validating: 100%|██████████| 938/938 [00:21<00:00, 43.48it/s]



Epoch 14:
Train Loss: 12.1175
Train Acc: 0.7114
Test Acc: 0.7136
Saved new best model


Validating: 100%|██████████| 938/938 [00:21<00:00, 43.31it/s]


Epoch 15:
Train Loss: 12.1149
Train Acc: 0.7155
Test Acc: 0.7136

Best Validation Accuracy: 0.7136


In [18]:
# достаем ученика
student = AdaBERT(vocab_size=len(tokenizer.vocab), max_layers=8).to('cuda')
student.load_state_dict(torch.load("best_adabert.pth"))


<All keys matched successfully>

In [ ]:
# выводим выбранную архитектуру
selected_layers = []
selected_ops = []

with torch.no_grad():
    layer_probs = F.softmax(student.layer_weights, dim=-1)
    for i, prob in enumerate(layer_probs):
        if prob > 0.1:
            selected_layers.append(i)
            op_idx = torch.argmax(student.op_weights[i]).item()
            op_name = list(SearchCell().ops.keys())[op_idx]
            selected_ops.append(op_name)

print("Лучшая архитектура:")
for layer, op in zip(selected_layers, selected_ops):
    print(f"  Layer {layer} → {op}")


Лучшая архитектура:
  Layer 1 → avg_pool
  Layer 2 → avg_pool
  Layer 5 → avg_pool
  Layer 6 → avg_pool
  Layer 7 → avg_pool


In [19]:
class FrozenSearchCell(nn.Module):
    def __init__(self, op, hidden_size):
        super().__init__()
        self.op = SearchCell(hidden_size).ops[op]

    def forward(self, x):
        return self.op(x)

class FrozenAdaBERT(nn.Module):
    def __init__(self, vocab_size, hidden_size=256, selected_layers=None, selected_ops=None):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.cells = nn.ModuleList([
            FrozenSearchCell(op, hidden_size) for op in selected_ops
        ])
        self.selected_layers = selected_layers
        self.classifier = nn.Linear(hidden_size, 2)

    def forward(self, input_ids):
        x = self.embedding(input_ids).transpose(1, 2)
        for cell in self.cells:
            x = cell(x)
        x = x.mean(dim=-1)
        return self.classifier(x)


In [20]:
frozen_model = FrozenAdaBERT(
    vocab_size=len(tokenizer.vocab),
    hidden_size=256,
    selected_layers=[3, 4, 5, 7],
    selected_ops=['avg_pool', 'avg_pool', 'avg_pool', 'avg_pool']
).to('cuda')

# Копируем веса
frozen_model.embedding.load_state_dict(student.embedding.state_dict())
frozen_model.classifier.load_state_dict(student.classifier.state_dict())

for i, layer in enumerate(frozen_model.cells):
    op_name = 'avg_pool'
    source = student.cells[selected_layers[i]].ops[op_name]
    layer.op.load_state_dict(source.state_dict())


In [21]:
per_device_train_batch_size = 8
per_device_eval_batch_size = 16
learning_rate = 0.003
optim_type = "adamw_torch"
lr_scheduler_type = "linear"
train_dataset = TextDataset(train_df['review'].values, train_df['sentiment'].values, tokenizer)
test_dataset = TextDataset(test_df['review'].values, test_df['sentiment'].values, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)
optimizer = torch.optim.AdamW(frozen_model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_dataset, batch_size=per_device_train_batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=per_device_eval_batch_size)

In [ ]:
# дообучим выбранную модель
total_start_time = time.time()

for epoch in range(3):
    epoch_start_time = time.time()
    frozen_model.train()
    train_acc = 0
    train_loss = 0

    for batch in train_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        logits = frozen_model(batch['input_ids'])
        loss = F.cross_entropy(logits, batch['labels'])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_acc += calculate_accuracy(logits, batch['labels'])
        train_loss += loss.item()

    avg_train_acc = train_acc / len(train_dataloader)
    avg_train_loss = train_loss / len(train_dataloader)

    # Валидация
    frozen_model.eval()
    test_acc = 0
    with torch.no_grad():
        for batch in test_dataloader:
            batch = {k: v.to('cuda') for k, v in batch.items()}
            logits = frozen_model(batch['input_ids'])
            test_acc += calculate_accuracy(logits, batch['labels'])

    avg_test_acc = test_acc / len(test_dataloader)
    epoch_time = time.time() - epoch_start_time

    print(f"\nEpoch {epoch+1}:")
    print(f"  Train Loss:     {avg_train_loss:.4f}")
    print(f"  Train Accuracy: {avg_train_acc:.4f}")
    print(f"  Test Accuracy:  {avg_test_acc:.4f}")
    print(f"  Time: {epoch_time:.2f} seconds")

# total_time = time.time() - total_start_time


Epoch 1:
  Train Loss:     0.3756
  Train Accuracy: 0.8401
  Test Accuracy:  0.8670
  Time: 68.22 seconds

Epoch 2:
  Train Loss:     0.2877
  Train Accuracy: 0.8813
  Test Accuracy:  0.8823
  Time: 67.12 seconds

Epoch 3:
  Train Loss:     0.2484
  Train Accuracy: 0.9007
  Test Accuracy:  0.8979
  Time: 67.24 seconds


In [ ]:
# тут сохраняем модель для сравнения
save_dir = "/content/drive/MyDrive/all_optimiz_adabert_imdb_dir"
os.makedirs(save_dir, exist_ok=True)

torch.save(frozen_model.state_dict(), os.path.join(save_dir, "frozen_adabert.pt"))
tokenizer.save_pretrained(save_dir)

('/content/drive/MyDrive/all_optimiz_adabert_imdb_dir/tokenizer_config.json',
 '/content/drive/MyDrive/all_optimiz_adabert_imdb_dir/special_tokens_map.json',
 '/content/drive/MyDrive/all_optimiz_adabert_imdb_dir/vocab.txt',
 '/content/drive/MyDrive/all_optimiz_adabert_imdb_dir/added_tokens.json',
 '/content/drive/MyDrive/all_optimiz_adabert_imdb_dir/tokenizer.json')

Итоговая точность на тестовом наборе составила 0.8979. Если поставить больше эпох на обучение, точность на трейне растет до 1, в то время как на тесте остается в районе 0.91. Я считаю, что в рамках нашей лучше модели(точность которой порядка 94% +- 5% это допустимо). Сама модель AdaBert обучается 1 эпоха ~ 67-90 секунд (в зависимоти от параметров), а классический bert-base-uncased одна эпоха обучается порядка часа. Вес модели adaBert +-30 МБ, а нашей лучшей bert порядка 400. Мне кажется в рамках данной задачи я сжал берт и получил неплохие результаты.

Есть предположение, что если повысить кол-ва эпох для поиска оптимальной архитектуры до 50-80, то это даст более хорошие результаты с точки зрения архитектуры, но прирост на качестве будет +-2-3%, так, по крайней мере, происходит в профильных статьях. К сожалению, у меня умирал колаб и не получилось прогнать на таком большом количестве эпох

Все технические моменты распишу в доке

P.S. Это самая успешная из реализованных мной, поэтому она заливается на гит